# SQLAlchemy ORM project

## Tasks

### Task 1. 



Create a connection to a database using `sqlalchemy` (save it to a `database.py` file).

In [1]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker


SQLALCHEMY_DATABASE_URL = 'postgresql://login:password.lab.karpov.courses:6432/startml'

engine = create_engine(SQLALCHEMY_DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

Base = declarative_base()


### Task 2. 

Describe `post` table in a `public` schema as a class (save it to a `table_post.py` file). 

In [3]:
from database import Base, SessionLocal
from sqlalchemy import Column, Integer, String


class Post(Base):
    __tablename__ = 'post'
    __table_args__ = {'schema': 'public'}

    id = Column(Integer, primary_key=True)
    text = Column(String)
    topic = Column(String)

### Task 3. 

Make a query to the `post` table in order to find the last 10 ids of the posts having a business topic. Print the result of the query as a list of ids. 

In [8]:
if __name__ == '__main__':
    session = SessionLocal()
    results = (
        session.query(Post)
        .filter(Post.topic == 'business')
        .order_by(Post.id.desc())
        .limit(10)
        .all()
    )

    lst = []
    for x in results:
        lst.append(x.id)
    print(lst)

### Task 4. 

Describe `user` table in a `public` schema as a class (save it to a `table_user.py` file). 

In [ ]:
class User(Base):
    __tablename__ = 'user'
    __table_args__ = {'schema': 'public'}

    id = Column(Integer, primary_key=True)
    gender = Column(Integer)
    age = Column(Integer)
    country = Column(String)
    city = Column(String)
    exp_group = Column(Integer)
    os = Column(String)
    source = Column(String)

### Task 5. 

Make a query to the `user` table in order to find the top number of users grouped by country and os. Print in descending order only those having more than 100 users.  

In [ ]:
if __name__ == '__main__':
    session = SessionLocal()
    query = (
        select(User.country, User.os, func.count(User.id).label('user_count'))
        .filter(User.exp_group == 3)
        .group_by(User.country, User.os)
        .having(func.count(User.id) > 100)
        .order_by(func.count(User.id).desc())
    )

    result = session.execute(query).fetchall()
    print(result)

### Task 6. 

Describe `feed_action` table in a `public` schema as a class (save it to a `table_feed.py` file).

In [ ]:
from database import Base
from sqlalchemy import Column, Integer, String, TIMESTAMP, ForeignKey
from table_user import User
from table_post import Post


class Feed(Base):
    __tablename__ = 'feed_action'
    __table_args__ = {'schema': 'public'}

    user_id = Column(Integer, ForeignKey(User.id), primary_key=True)
    post_id = Column(Integer, ForeignKey(Post.id), primary_key=True)
    action = Column(String)
    time = Column(TIMESTAMP)

### Task 7. 

Describe all three tables as classes using pydantic `BaseModel` as a parent class (save it to a `schema.py` file).

In [ ]:
import datetime
from pydantic import BaseModel


class UserGet(BaseModel):
    id: int
    gender: int
    age: int
    country: str
    city: str
    exp_group:int
    os: str
    source: str

    class Config:
        orm_mode = True


class PostGet(BaseModel):
    id: int
    text: str
    topic: str

    class Config:
        orm_mode = True


class FeedGet(BaseModel):
    user_id: int
    post_id: int
    action: str
    time: datetime.datetime

    class Config:
        orm_mode = True


### Task 8. 

Code `GET /user/{id}` and `GET /post/{id}` endpoints using FastAPI (save it to a `app.py` file).

In [ ]:
from fastapi import FastAPI, HTTPException
from fastapi.params import Depends
from sqlalchemy.orm import Session

from database import SessionLocal
from schema import UserGet, PostGet
from table_user import User
from table_post import Post


# task 8
app = FastAPI()


def get_db():
    with SessionLocal() as db:
        return db


@app.get('/user/{id}', response_model=UserGet)
def get_user(id: int, db: Session = Depends(get_db)):
    user = db.query(User).filter(User.id == id).one_or_none()

    if not user:
        raise HTTPException(status_code=404, detail='User not found! Try another id.')
    return user


@app.get('/post/{id}', response_model=PostGet)
def get_post(id: int, db: Session = Depends(get_db)):
    post = db.query(Post).filter(Post.id == id).one_or_none()

    if not post:
        raise HTTPException(status_code=404, detail='Post not found! Try another id.')
    return post


### Task 9. 

Code `GET /user/{id}/feed` and `GET /post/{id}/feed` endpoints using FastAPI (save it to a app.py file).

In [ ]:
from fastapi import FastAPI, HTTPException
from fastapi.params import Depends
from sqlalchemy.orm import Session

from database import SessionLocal
from typing import List
from schema import UserGet, PostGet, FeedGet
from table_user import User
from table_post import Post
from table_feed import Feed


# task 8
app = FastAPI()


def get_db():
    with SessionLocal() as db:
        return db


@app.get('/user/{id}', response_model=UserGet)
def get_user(id: int, db: Session = Depends(get_db)):
    user = db.query(User).filter(User.id == id).one_or_none()

    if not user:
        raise HTTPException(status_code=404, detail='User not found! Try another id.')
    return user


@app.get('/post/{id}', response_model=PostGet)
def get_post(id: int, db: Session = Depends(get_db)):
    post = db.query(Post).filter(Post.id == id).one_or_none()

    if not post:
        raise HTTPException(status_code=404, detail='Post not found! Try another id.')
    return post


# task 9
@app.get('/user/{id}/feed', response_model=List[FeedGet])
def get_user_feed(id: int, limit: int = 10, db: Session = Depends(get_db)):
    feed = db.query(Feed).join(User).filter(Feed.user_id == id).order_by(Feed.time.desc()).limit(limit).all()

    if not feed:
        raise HTTPException(status_code=200, detail={'message': 'User not found! Try another id.', 'data': []})
    return feed


@app.get('/post/{id}/feed', response_model=List[FeedGet])
def get_user_feed(id: int, limit: int = 10, db: Session = Depends(get_db)):
    feed = db.query(Feed).join(Post).filter(Feed.post_id == id).order_by(Feed.time.desc()).limit(limit).all()

    if not feed:
        raise HTTPException(status_code=200, detail={'message': 'Post not found! Try another id.', 'data': []})
    return feed


### Task 10. 

Add `relationships` to your classes. Update `table_feed.py` and `schema.py` files. 

Updated `table_feed.py` file:

In [ ]:
from sqlalchemy.orm import relationship

from database import Base
from sqlalchemy import Column, Integer, String, TIMESTAMP, ForeignKey
from table_user import User
from table_post import Post


# task 6
class Feed(Base):
    __tablename__ = 'feed_action'
    __table_args__ = {'schema': 'public'}

    user_id = Column(Integer, ForeignKey(User.id), primary_key=True)
    post_id = Column(Integer, ForeignKey(Post.id), primary_key=True)
    action = Column(String)
    time = Column(TIMESTAMP)

    # task 10
    user = relationship('User')
    post = relationship('Post')

Updated `schema.py` file:

In [ ]:
import datetime
from pydantic import BaseModel


# task 7
class UserGet(BaseModel):
    id: int
    gender: int
    age: int
    country: str
    city: str
    exp_group:int
    os: str
    source: str

    class Config:
        orm_mode = True


class PostGet(BaseModel):
    id: int
    text: str
    topic: str

    class Config:
        orm_mode = True


class FeedGet(BaseModel):
    user_id: int
    post_id: int
    action: str
    time: datetime.datetime

    #task 10
    user: UserGet
    post: PostGet

    class Config:
        orm_mode = True

### Task 11. 

Code `GET /post/recommendations/` endpoints using FastAPI (save it to a app.py file).

In [ ]:
from fastapi import FastAPI, HTTPException
from fastapi.params import Depends
from sqlalchemy import func
from sqlalchemy.orm import Session

from database import SessionLocal
from typing import List
from schema import UserGet, PostGet, FeedGet
from table_user import User
from table_post import Post
from table_feed import Feed


# task 8
app = FastAPI()


def get_db():
    with SessionLocal() as db:
        return db


@app.get('/user/{id}', response_model=UserGet)
def get_user(id: int, db: Session = Depends(get_db)):
    user = db.query(User).filter(User.id == id).one_or_none()

    if not user:
        raise HTTPException(status_code=404, detail='User not found! Try another id.')
    return user


@app.get('/post/{id}', response_model=PostGet)
def get_post(id: int, db: Session = Depends(get_db)):
    post = db.query(Post).filter(Post.id == id).one_or_none()

    if not post:
        raise HTTPException(status_code=404, detail='Post not found! Try another id.')
    return post


# task 9
@app.get('/user/{id}/feed', response_model=List[FeedGet])
def get_user_feed(id: int, limit: int = 10, db: Session = Depends(get_db)):
    feed = db.query(Feed).join(User).filter(Feed.user_id == id).order_by(Feed.time.desc()).limit(limit).all()

    if not feed:
        raise HTTPException(status_code=200, detail={'message': 'User not found! Try another id.', 'data': []})
    return feed


@app.get('/post/{id}/feed', response_model=List[FeedGet])
def get_user_feed(id: int, limit: int = 10, db: Session = Depends(get_db)):
    feed = db.query(Feed).join(Post).filter(Feed.post_id == id).order_by(Feed.time.desc()).limit(limit).all()

    if not feed:
        raise HTTPException(status_code=200, detail={'message': 'Post not found! Try another id.', 'data': []})
    return feed


# task 11
@app.get('/post/recommendations/', response_model=List[PostGet])
def get_post_recommendations(id: int = None, limit: int = 10, db: Session = Depends(get_db)):
    posts = db.query(
        Post.id,
        Post.text,
        Post.topic
        #,func.count(Feed.post_id).label('likes_count')
    ) \
        .join(Feed, Post.id == Feed.post_id) \
        .filter(Feed.action == 'like') \
        .group_by(Post.id, Post.text, Post.topic) \
        .order_by(func.count(Feed.post_id).desc()) \
        .limit(limit) \
        .all()
    return posts
